In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

### Skip this cell if you already have a chunk of the transactions data.

In [2]:
file_name = '../data/transactions.csv'
chunksize = 10 ** 5
chunk_iter = pd.read_csv(file_name, chunksize=chunksize)
chunk1 = chunk_iter.next()
chunk1.to_csv('../data/transactions_chunk1.csv', index=False)

In [3]:
lines = sc.textFile('../data/transactions_chunk1.csv')

In [4]:
header = lines.first()
lines = lines.filter(lambda row: row != header)

In [5]:
# Functions for parsing the strings from the CSV file

def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None

def toFloatSafe(inval):
  try:
    return float(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf8').split(',')
  return Row(
    int(r[0]),         # Don't want this column to be nullable
    toLongSafe(r[1]),
    toLongSafe(r[2]),
    toLongSafe(r[3]),
    toLongSafe(r[4]),
    toLongSafe(r[5]),
    toTimeSafe(r[6]),
    toFloatSafe(r[7]),
    r[8],
    toLongSafe(r[9]),
    toFloatSafe(r[10]))  

In [6]:
transactions_schema = StructType([
    StructField("id", LongType(), False),
    StructField("chain", LongType(), True),
    StructField("dept", LongType(), True),
    StructField("category", LongType(), True),
    StructField("company", LongType(), True),
    StructField("brand", LongType(), True),
    StructField("date", DateType(), True),
    StructField("productsize", DoubleType(), True),
    StructField("purchasemeasure", StringType(), True),
    StructField("purchasequantity", LongType(), True),
    StructField("purchaseamount", DoubleType(), True)
    ])

In [7]:
rowRDD = lines.map(lambda p: stringToPost(p))

In [8]:
transactions = sqlContext.createDataFrame(rowRDD, transactions_schema)

In [9]:
transactions.printSchema()

root
 |-- id: long (nullable = false)
 |-- chain: long (nullable = true)
 |-- dept: long (nullable = true)
 |-- category: long (nullable = true)
 |-- company: long (nullable = true)
 |-- brand: long (nullable = true)
 |-- date: date (nullable = true)
 |-- productsize: double (nullable = true)
 |-- purchasemeasure: string (nullable = true)
 |-- purchasequantity: long (nullable = true)
 |-- purchaseamount: double (nullable = true)



In [10]:
transactions.show()

+-----+-----+----+--------+----------+-----+----------+-----------+---------------+----------------+--------------+
|   id|chain|dept|category|   company|brand|      date|productsize|purchasemeasure|purchasequantity|purchaseamount|
+-----+-----+----+--------+----------+-----+----------+-----------+---------------+----------------+--------------+
|86246|  205|   7|     707|1078778070|12564|2012-03-02|       12.0|             OZ|               1|          7.59|
|86246|  205|  63|    6319| 107654575|17876|2012-03-02|       64.0|             OZ|               1|          1.59|
|86246|  205|  97|    9753|1022027929|    0|2012-03-02|        1.0|             CT|               1|          5.99|
|86246|  205|  25|    2509| 107996777|31373|2012-03-02|       16.0|             OZ|               1|          1.99|
|86246|  205|  55|    5555| 107684070|32094|2012-03-02|       16.0|             OZ|               2|         10.38|
|86246|  205|  97|    9753|1021015020|    0|2012-03-02|        1.0|     

# Training data and testing data

In [11]:
train = pd.read_csv('../data/trainHistory.csv')
offers = pd.read_csv('../data/offers.csv')

In [12]:
# Add department to offers
dept = []
for i in range(offers.shape[0]):
    str_category = str(offers['category'].iloc[i])
    if len(str_category) == 4:
        dept.append(str_category[:2])
    else:
        dept.append(str_category[:1])
        
offers['dept'] = dept
offers['dept'] = pd.to_numeric(offers['dept'])

In [13]:
train = pd.merge(train, offers, how='left', on=['offer'])

In [14]:
# Rename columns
train.columns = ['id', 'chain', 'offer', 'market', 'repeattrips', 'repeater', 'offerdate', 
                 'offer_category', 'quantity', 'offer_company', 'offervalue', 'offer_brand',
                 'offer_dept']

In [15]:
# Reencode the target variable
train['repeater'] = np.where(train['repeater'] == 't', '1', '0')
train['repeater'] = pd.to_numeric(train['repeater'])

In [16]:
train['offerdate'] = train['offerdate'].astype(str)

In [17]:
train_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("repeattrips", LongType(), True),
    StructField("repeater", IntegerType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True)
    ])

In [18]:
train_df = sqlContext.createDataFrame(train, train_schema)

In [19]:
test = pd.read_csv('../data/testHistory.csv')
test = pd.merge(test, offers, how='left', on=['offer'])

In [20]:
# Rename columns
test.columns = ['id', 'chain', 'offer', 'market', 'offerdate', 'offer_category', 'quantity', 
                'offer_company', 'offervalue', 'offer_brand', 'offer_dept']

In [21]:
test['offerdate'] = test['offerdate'].astype(str)

In [22]:
test_schema = StructType([
    StructField("id", LongType(), False),
    StructField("offer_chain", LongType(), True),
    StructField("offer", LongType(), True),
    StructField("market", LongType(), True),
    StructField("offerdate", StringType(), True),
    StructField("offer_category", LongType(), True),
    StructField("quantity", LongType(), True),
    StructField("offer_company", LongType(), True),
    StructField("offervalue", DoubleType(), True),
    StructField("offer_brand", LongType(), True),
    StructField("offer_dept", LongType(), True)
    ])

In [23]:
test_df = sqlContext.createDataFrame(test, test_schema)

In [24]:
# This function converts the string cell into a date:
stringToDate = F.udf(lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())

In [25]:
# Convert offerdate to date type
train_df = train_df.withColumn('offerdate', stringToDate(F.col('offerdate')))
test_df = test_df.withColumn('offerdate', stringToDate(F.col('offerdate')))

In [26]:
train_df = train_df.withColumnRenamed('repeater', 'label')

# Feature Engineering

### Get number of refund transactions

In [27]:
id_stats = transactions.select('id', 'purchasequantity', 'purchaseamount',
                              F.when(transactions['purchaseamount'] < 0, 1).otherwise(0).alias('return'))\
                      .withColumn('1', F.lit(1))
id_stats = id_stats.groupBy('id').agg(F.sum('1').alias('total_trans'), 
                                      F.sum('return').alias('total_returns'),
                                      F.avg('purchasequantity').alias('avg_purchasequantity'),
                                      F.avg('purchaseamount').alias('avg_purchaseamount'))
id_stats.show()

+--------+-----------+-------------+--------------------+------------------+
|      id|total_trans|total_returns|avg_purchasequantity|avg_purchaseamount|
+--------+-----------+-------------+--------------------+------------------+
|18470775|        350|            8|                1.34|3.1511428571428635|
|14723452|        755|            5|  1.2264900662251657| 4.073933774834406|
|15738658|         39|            0|   1.205128205128205| 4.780000000000001|
|17552659|        591|            4|  1.5245346869712353| 4.931455160744474|
|12996040|        326|            5|  1.2331288343558282|4.7280981595092095|
|16078766|        966|           54|   1.818840579710145| 5.298498964803263|
|18249735|       1557|           60|  1.2594733461785486|  4.13752087347458|
|14989775|        614|           39|   1.231270358306189|3.8022475570032475|
|15073302|        526|           38|  1.3669201520912548| 4.640133079847899|
|16075389|        591|           32|  1.9593908629441625| 7.553604060913664|

In [28]:
# Rename id column of returns table to avoid ambiguous column names
id_stats = id_stats.withColumnRenamed('id', 'returns_id')

In [29]:
train_df = train_df.join(id_stats, train_df['id'] == id_stats['returns_id'], 'left')
test_df = test_df.join(id_stats, test_df['id'] == id_stats['returns_id'], 'left')

In [30]:
train_df = train_df.select('offer', 'offer_category', 'offer_brand', 'offer_company', 'offervalue', 
                           'total_trans', 'total_returns', 'avg_purchasequantity', 'avg_purchaseamount', 'label')
test_df = test_df.select('offer', 'offer_category', 'offer_brand', 'offer_company', 'offervalue', 
                         'total_trans', 'total_returns', 'avg_purchasequantity', 'avg_purchaseamount', 'id')

In [31]:
# Get average value for imputations
avg_total_trans = id_stats.agg(F.avg(F.col('total_trans'))).first()[0]
avg_total_returns = id_stats.agg(F.avg(F.col('total_returns'))).first()[0]
avg_avg_purchasequantity = id_stats.agg(F.avg(F.col('avg_purchasequantity'))).first()[0]
avg_avg_purchaseamount = id_stats.agg(F.avg(F.col('avg_purchaseamount'))).first()[0]

# Impute missing data
impute_dict = {'total_trans': avg_total_trans, 'total_returns': avg_total_returns,
               'avg_purchasequantity': avg_avg_purchasequantity,
               'avg_purchaseamount': avg_avg_purchaseamount}
train_df = train_df.fillna(impute_dict)
test_df = test_df.fillna(impute_dict)


# Create pipeline

In [32]:
# Get all the offers, categories, brands and companies from both training and test
# Then create a StringIndexer to map categorical values to numeric labels
cols_to_index = ['offer', 'offer_category', 'offer_brand', 'offer_company']
indexers = []
for col in cols_to_index:
    union_df = train_df.select(col).union(test_df.select(col))
    indexers.append(StringIndexer().setInputCol(col).setOutputCol(col+'idx').fit(union_df))

In [33]:
# One-hot-encoding of categorical variables
encoders = []
for col in cols_to_index:
    encoders.append(OneHotEncoder().setInputCol(col+'idx').setOutputCol(col+'_encoded').setDropLast(False))

In [34]:
# Assemble features
inputCols = ['offervalue', 'offer_encoded', 'offer_category_encoded', 'offer_brand_encoded', 'offer_company_encoded', 
             'total_trans', 'total_returns', 'avg_purchasequantity', 'avg_purchaseamount']
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')

In [ ]:
# Logistic regression
lr = LogisticRegression(maxIter=10, standardization=True)

In [35]:
# Pipeline
stages = indexers + encoders + [assembler, lr]
pipeline = Pipeline(stages=stages)

# Cross-validation

In [36]:
regularization = [0.0001, 0.001]   # for testing purposes
#regularization = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
kfolds = 2

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, regularization) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=kfolds)  # use 3+ folds in practice

In [37]:
train_df.cache()
test_df.cache()

DataFrame[offer: bigint, offer_category: bigint, offer_brand: bigint, offer_company: bigint, offervalue: double, total_trans: bigint, total_returns: bigint, avg_purchasequantity: double, avg_purchaseamount: double, id: bigint]

In [38]:
cvModel = crossval.fit(train_df)

In [39]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(cvModel.bestModel.stages[-1].coefficients))
print("Intercept: " + str(cvModel.bestModel.stages[-1].intercept))

Coefficients: [-0.0429443394893,-0.120828064549,0.0,0.0,0.0,0.211433225354,0.0637029341056,0.192683530963,0.0,-0.240026654901,0.0160405151517,0.0,0.127899969392,-0.198888647052,0.282359546423,-0.152730849875,0.0,0.0,0.0399239681762,0.0,0.0,0.081020743848,0.0,-0.219622642058,-0.123039556646,0.0,-0.109590355519,-0.0843160148191,-0.0661700731342,0.0,0.0,-0.127940962218,-0.0986583139025,-0.121933421527,-0.192675565725,-0.0899414243517,-0.0705857787871,-0.0473126533044,0.0,-0.120828064549,0.0,0.23650271708,0.211433225354,0.0613482528862,-0.232047234628,0.0,0.0,0.117857839674,0.0160405151517,-0.0873028040284,-0.149545108453,-0.118094349399,0.0,-0.213903048893,0.0,0.0,-0.0986583139025,-0.0473126533044,0.0,-0.120828064549,0.0,0.211433225354,0.0613482528862,0.192683530963,-0.232047234628,0.0,0.0,-0.178737903433,0.117857839674,0.0160405151517,-0.0873028040284,0.282359546423,-0.118094349399,0.0,0.0,0.0,-0.0986583139025,0.0,-0.120828064549,-0.0329635068416,0.0,0.211433225354,0.192683530963,-0.2320

# Predicting on training data 

In [40]:
train_pred = cvModel.transform(train_df)

In [41]:
train_pred.select('probability', 'prediction').show()

+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.72003434577649...|       0.0|
|[0.72003434577649...|       0.0|
|[0.73328529723190...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82166951058277...|       0.0|
|[0.72003434577649...|       0.0|
|[0.72927779755293...|       0.0|
|[0.82918143595882...|       0.0|
|[0.82918143595882...|       0.0|
|[0.82166951058277...|       0.0|
|[0.82353291521212...|       0.0|
|[0.82166951058277...|       0.0|
+--------------------+----------+
only showing top 20 rows



# Predicting on test data

In [42]:
test_pred = cvModel.transform(test_df)

In [43]:
test_pred.select('probability', 'prediction').show()

+--------------------+----------+
|         probability|prediction|
+--------------------+----------+
|[0.76479254131548...|       0.0|
|[0.75786544152271...|       0.0|
|[0.77327570477242...|       0.0|
|[0.74584921916645...|       0.0|
|[0.74175755178399...|       0.0|
|[0.74584921916645...|       0.0|
|[0.74989791590015...|       0.0|
|[0.74175755178399...|       0.0|
|[0.75786544152271...|       0.0|
|[0.75786544152271...|       0.0|
|[0.74989791590015...|       0.0|
|[0.75786544152271...|       0.0|
|[0.74584921916645...|       0.0|
|[0.74584921916645...|       0.0|
|[0.74584921916645...|       0.0|
|[0.74989791590015...|       0.0|
|[0.77327570477242...|       0.0|
|[0.74989791590015...|       0.0|
|[0.74584921916645...|       0.0|
|[0.75786544152271...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [44]:
test_id = test_df.select('id').toPandas()
test_prob = test_pred.select('probability').toPandas()

In [45]:
# I didn't know how to access the vector inside the probability column.
# To work around this, I looped through the probability column and used the indexing operations to pull out
# the individual probabilities.
prob = []
for i in range(test_prob.shape[0]):
    prob.append(test_prob['probability'][i][1])
test_prob['repeatProbability'] = prob

In [46]:
test_final_pred = pd.merge(test_id, test_prob, left_index=True, right_index=True)
test_final_pred.drop('probability', 1, inplace=True)

In [47]:
test_final_pred.head()

,id,repeatProbability
0,18470775,0.235207
1,86880128,0.242135
2,87553372,0.226724
3,98483695,0.254151
4,101001592,0.258242


In [48]:
test_final_pred['repeatProbability'].describe()

count    151484.000000
mean          0.252107
std           0.017777
min           0.170819
25%           0.250102
50%           0.254151
75%           0.254151
max           0.447843
Name: repeatProbability, dtype: float64

In [49]:
# Write the test file to csv
#test_final_pred.to_csv('submission.csv', index=False)